In [106]:
import matplotlib.pyplot as plt
import numpy as np
from numpy.linalg import norm
from scipy.integrate import odeint
import pprint
from ipywidgets import interact, interactive, fixed, interact_manual, jslink
import ipywidgets as widgets
import ipyvolume as ipv
import ipyvolume.pylab as p3
import traitlets
import time, sys
from IPython.core.display import clear_output
from random import random

In [201]:
# define the atoms in the chain (this test will only use a 3 atom chain)
r1 = (-1, 2, 0)
r2 = (-.2, 0, 0)
r3 = (.2, 0, 0)
r4 = (1, 2, 0)

floaters = np.hstack((r1, r2, r3, r4))

m = 4             # total atoms in the chain

In [202]:
# define the number of floating atoms
#floaters = np.zeros(4*3) # set number of floaters and multiply by 3 to have 3 coords for each
#n = len(floaters)
#m = n/3                    # total atoms in the chain

# set the positions of the floating atoms
#count = 0
#k = 1.7
#for i in range(0,n,3):
#    floaters[i] = random()
#    floaters[i+1] = random()
#    floaters[i+2] = random()
#    count += 1

In [203]:
# define parameters
beta = 20

In [229]:
# define time interval
t = np.linspace(0,.03,500)

In [230]:
# bending force DE system
def bending_force(floaters, t):
    
    current_bending_force = np.zeros((int(m),3))
    total_bending_force = np.zeros(int(m))
    
    # loop for computing bending force
    for i in range(1,int(m-1)):          # loop through each atom not on the end of the chain and compute the bending force on it and its neighbors
        
        # define the pieces of the bending energy equation
        sx_n = floaters[i*3] - floaters[(i-1)*3]
        sx_n1 = floaters[(i+1)*3] - floaters[i*3]

        sy_n = floaters[i*3 + 1] - floaters[(i-1)*3 + 1]
        sy_n1 = floaters[(i+1)*3 + 1] - floaters[i*3 + 1]

        sz_n = floaters[i*3 + 2] - floaters[(i-1)*3 + 2]
        sz_n1 = floaters[(i+1)*3 + 2] - floaters[i*3 + 2]

        A = norm((sx_n, sy_n, sz_n))*norm((sx_n1, sy_n1, sz_n1))

        B = np.dot((sx_n, sy_n, sz_n),(sx_n1, sy_n1, sz_n1))

        partial_common = .5/A
        
        # partials of A wrt x for atom n, n-1 and n+1, respectively
        Axn = partial_common*(2*(sx_n)*(sx_n1**2) - 2*(sx_n1)*(sx_n**2) + 2*(sx_n)*(sy_n1**2) + 2*(sx_n)*(sz_n1**2) - 2*(sx_n1)*(sy_n**2) - 2*(sx_n1)*(sz_n**2))
        Axn0 = partial_common*(-2*(sx_n)*(sx_n1**2) - 2*(sx_n)*(sy_n1**2) - 2*(sx_n)*(sz_n1**2))
        Axn1 = partial_common*(2*(sx_n**2)*(sx_n1) + 2*(sy_n**2)*(sx_n1) + 2*(sz_n**2)*(sx_n1))
        
        # partials of A wrt y for atom n, n-1 and n+1, respectively
        Ayn = partial_common*(-2*(sx_n**2)*(sy_n1) + 2*(sy_n)*(sx_n1**2) + 2*(sy_n)*(sy_n1**2) - 2*(sy_n**2)*(sy_n1) + 2*(sy_n)*(sz_n1**2) - 2*(sz_n**2)*(sy_n1))
        Ayn0 = partial_common*(-2*(sy_n)*(sx_n1**2) - 2*(sy_n)*(sy_n1**2) - 2*(sy_n)*(sz_n1**2))
        Ayn1 = partial_common*(2*(sy_n**2)*(sy_n1) + 2*(sz_n**2)*(sy_n1) + 2*(sx_n**2)*(sy_n1))
        
        # partials of A wrt z for atom n, n-1 and n+1, respectively
        Azn = partial_common*(-2*(sx_n**2)*(sz_n1) - 2*(sy_n**2)*(sz_n1) - 2*(sz_n**2)*(sz_n1) + 2*(sz_n)*(sz_n1**2) + 2*(sz_n)*(sx_n1**2) + 2*(sz_n)*(sy_n1**2))
        Azn0 = partial_common*(-2*(sz_n)*(sx_n1**2) -2*(sz_n)*(sy_n1**2) - 2*(sz_n)*(sz_n1**2))
        Azn1 = partial_common*(2*(sz_n1)*(sx_n**2) + 2*(sz_n1)*(sy_n**2) + 2*(sz_n1)*(sz_n**2))
        
        # partials of B wrt x for atom n, n-1 and n+1, respectively
        Bxn = floaters[(i+1)*3] - 2*floaters[i*3] + floaters[(i-1)*3]
        Bxn0 = -floaters[(i+1)*3] + floaters[i*3]
        Bxn1 = floaters[i*3] - floaters[(i-1)*3]

        # partials of B wrt y for atom n, n-1 and n+1, respectively
        Byn = floaters[(i+1)*3 + 1] - 2*floaters[i*3 + 1] + floaters[(i-1)*3 + 1]
        Byn0 = -floaters[(i+1)*3 + 1] + floaters[i*3 + 1]
        Byn1 = floaters[i*3 + 1] - floaters[(i-1)*3 + 1]
        
        # partials of B wrt z for atom n, n-1 and n+1, respectively
        Bzn = floaters[(i+1)*3 + 2] - 2*floaters[i*3 + 2] + floaters[(i-1)*3 + 2]
        Bzn0 = -floaters[(i+1)*3 + 2] + floaters[i*3 + 2]
        Bzn1 = floaters[i*3 + 2] - floaters[(i-1)*3 + 2]
        
        try:
            Eb_common = 4*beta/((A+B)**2)                   # compute common part of bending energy gradient
            if (np.isnan(Eb_common)  or np.isinf(Eb_common) or np.isnan(A)):
                raise ValueError('Divide by zero')
                raise Exception
            
        except Exception as error:
            print("Caught exception: ", error)
            continue
            
        current_bending_force[i,0] += Eb_common*(A*Bxn - B*Axn)
        current_bending_force[i,1] += Eb_common*(A*Byn - B*Ayn)
        current_bending_force[i,2] += Eb_common*(A*Bzn - B*Azn)
            
        current_bending_force[i-1,0] += Eb_common*(A*Bxn0 - B*Axn0)
        current_bending_force[i-1,1] += Eb_common*(A*Byn0 - B*Ayn0)
        current_bending_force[i-1,2] += Eb_common*(A*Bzn0 - B*Azn0)
            
        current_bending_force[i+1,0] = Eb_common*(A*Bxn1 - B*Axn1)
        current_bending_force[i+1,1] = Eb_common*(A*Byn1 - B*Ayn1)
        current_bending_force[i+1,2] = Eb_common*(A*Bzn1 - B*Azn1)

    total_bending_force = np.hstack(current_bending_force)
    return total_bending_force

In [231]:
%%time

# solve the system of ODEs to compute changing positions as a result of the bending force acting on the chain
gFlow = odeint(bending_force, floaters, t, atol=1.4e-10)

CPU times: user 124 ms, sys: 2.75 ms, total: 127 ms
Wall time: 126 ms


/anaconda/lib/python3.6/site-packages/scipy/integrate/odepack.py:218: ODEintWarning: Repeated convergence failures (perhaps bad Jacobian or tolerances). Run with full_output = 1 to get quantitative information.
  warnings.warn(warning_msg, ODEintWarning)


In [232]:
print(gFlow)

[[ -1.00000000e+000   2.00000000e+000   0.00000000e+000 ...,
    1.00000000e+000   2.00000000e+000   0.00000000e+000]
 [ -1.00102233e+000   1.99959046e+000   0.00000000e+000 ...,
    1.00102233e+000   1.99959046e+000   0.00000000e+000]
 [ -1.00204242e+000   1.99918061e+000   0.00000000e+000 ...,
    1.00204242e+000   1.99918061e+000   0.00000000e+000]
 ..., 
 [  0.00000000e+000   0.00000000e+000   2.52961611e-320 ...,
    0.00000000e+000   0.00000000e+000   3.60739443e-313]
 [  0.00000000e+000   0.00000000e+000   2.75859610e-313 ...,
    0.00000000e+000   0.00000000e+000   1.48539862e-313]
 [  0.00000000e+000   0.00000000e+000   2.52961611e-320 ...,
    0.00000000e+000   0.00000000e+000   2.40313530e-320]]


## Checking Energy Behavior

In [233]:
def bending_energy(row, atom):
    
    # if-else statement to check if I am looking at an atom on the end, 
    # an atom one from the end or an atom in the middle of the chain
    
    r1 = gFlow[row, atom*3-3:atom*3]
    r2 = gFlow[row, atom*3:atom*3+3]
    r3 = gFlow[row, atom*3+3:atom*3+6]
    
    s1 = (r2[0] - r1[0], r2[1] - r1[1], r2[2] - r1[2])
    s2 = (r3[0] - r2[0], r3[1] - r2[1], r3[2] - r2[2])
        
    A = norm(s1)*norm(s2)
    B = np.dot(s1, s2)
        
    E_b = (2*beta*(A - B))/(A + B)
    
    return E_b

In [234]:
# bending_energies contains the individual energy of each angle at each time step, 
# total_energy is the energy of the system at each time step
bending_energies = np.zeros((len(gFlow), int(m)-1))
total_energy = np.zeros(len(gFlow))

for i in range(len(gFlow)):
    for j in range(1, int(m)-1):     # only need to compute energy of each angle, not each atom
        
        # call the function above to compute the energy
        bending_energies[i][j] = bending_energy(i, j)
        
        # increase total system energy by the energy of the newly computed angle
        total_energy[i] += bending_energies[i][j]
        
print(bending_energies, total_energy, sep='\n\n')

[[  0.          18.33494523  18.33494523]
 [  0.          18.29459639  18.29459639]
 [  0.          18.25439188  18.25439188]
 ..., 
 [  0.                  nan          nan]
 [  0.                  nan          nan]
 [  0.                  nan          nan]]

[ 36.66989047  36.58919278  36.50878375  36.42866182  36.34882534
  36.26927275  36.19000251  36.11101307  36.0323029   35.95387046
  35.87571425  35.79783276  35.72022452  35.64288803  35.56582183
  35.48902446  35.41249449  35.33623048  35.26023099  35.18449462
  35.10901997  35.03380564  34.95885026  34.88415244  34.80971083
  34.73552407  34.66159083  34.58790976  34.51447956  34.44129889
  34.36836647  34.295681    34.22324119  34.15104576  34.07909345
  34.00738301  33.93591319  33.86468274  33.79369043  33.72293506
  33.65241539  33.58213023  33.51207838  33.44225865  33.37266987
  33.30331086  33.23418047  33.16527752  33.09660089  33.02814943
  32.95992201  32.89191752  32.82413482  32.75657282  32.68923042
  32.62210653

/Users/christopher/.anaconda/ipykernel/ipykernel_launcher.py:16: RuntimeWarning: invalid value encountered in double_scalars
  app.launch_new_instance()
/Users/christopher/.anaconda/ipykernel/ipykernel_launcher.py:16: RuntimeWarning: divide by zero encountered in double_scalars
  app.launch_new_instance()
/Users/christopher/.anaconda/ipykernel/ipykernel_launcher.py:13: RuntimeWarning: invalid value encountered in double_scalars
  del sys.path[0]


## Create Animation to Check that The Force is Correct

In [235]:
# define arrays of x, y and z coordinates to create an animated scatter plot
x = np.zeros((len(t),int(m)))
y = np.zeros((len(t),int(m)))
z = np.zeros((len(t),int(m)))
for i in range(len(t)):
    for j in range(int(m)):
        x[i,j] = gFlow[i,j*3]
        y[i,j] = gFlow[i,j*3+1]
        z[i,j] = gFlow[i,j*3+2]

In [236]:
%%time
%matplotlib inline

# create an ipyvolume.pylab figure for the simulation
p3.figure()

# plot a scatterplot of the arrays defined above in the figure
s1 = p3.scatter(x, y, z, marker='sphere', connected=True)
s2 = p3.plot(x, y, z)

# set the x, y and z limits of the figure
p3.xlim(-10,10)
p3.ylim(-10,10)
p3.zlim(0,2)

# add animation controls to the figure
p3.animation_control([s1, s2], interval=400)

# add sliders for parameters
beta_slider = widgets.IntSlider(min=0, max=15, step=1, value=beta, description='beta', continuous_update=False)

# create handler functions to change the values of the parameters globally when the sliders change
def on_beta_slider_change(change):
    global beta
    beta = change.new

# set the sliders to call their handler functions any time the sliders are moved
beta_slider.observe(on_beta_slider_change, names='value')

# display the VBox beneath the total system energy plot
label = widgets.Label('Parameter sliders:')
parameters = widgets.VBox([ipv.gcc(), label, beta_slider])
display(parameters)

VBox(children=(VBox(children=(Figure(animation=400.0, animation_exponent=1.0, camera_center=[0.0, 0.0, 0.0], height=500, matrix_projection=[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], matrix_world=[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], scatters=[Scatter(color_selected=array('white',
      dtype='<U5'), connected=True, geo='sphere', size=array(2), size_selected=array(2.6), x=array([[-1.        , -0.2       ,  0.2       ,  1.        ],
       [-1.00102233, -0.19897767,  0.19897767,  1.00102233],
       [-1.00204242, -0.19795758,  0.19795758,  1.00204242],
       ..., 
       [ 0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ]]), y=array([[  2.00000000e+000,   0.00000000e+000,   0.00000000e+000,
          2.00000000e+000],
       [  1.99959046e+000,   4.09540058e-004,   4.09540058e-004,
          1.99959046e+000],
       [  1.99918061e+000,   8.19389420e-004,   8.19389420e-004,
          1.99918061e+000],
       ..., 
       [  0.00000000e+000,   0.00000000e+000,   0.00000000e+000,
          0.00000000e+000],
       [  0.00000000e+000,   1.63041663e-322,   0.00000000e+000,
          0.00000000e+000],
       [  0.00000000e+000,   0.00000000e+000,   0.00000000e+000,
          0.00000000e+000]]), z=array([[  0.00000000e+000,   0.00000000e+000,   0.00000000e+000,
          0.00000000e+000],
       [  0.00000000e+000,   0.00000000e+000,   0.00000000e+000,
          0.00000000e+000],
       [  0.00000000e+000,   0.00000000e+000,   0.00000000e+000,
          0.00000000e+000],
       ..., 
       [  2.52961611e-320,   3.03553933e-320,   2.52961611e-320,
          3.60739443e-313],
       [  2.75859610e-313,   6.36600002e-314,   1.39128886e-320,
          1.48539862e-313],
       [  2.52961611e-320,   1.27319904e-313,   6.36600002e-314,
          2.40313530e-320]])), Scatter(color_selected=None, connected=True, size=array(1), size_selected=array(1), visible_lines=True, visible_markers=False, x=array([[-1.        , -0.2       ,  0.2       ,  1.        ],
       [-1.00102233, -0.19897767,  0.19897767,  1.00102233],
       [-1.00204242, -0.19795758,  0.19795758,  1.00204242],
       ..., 
       [ 0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ]]), y=array([[  2.00000000e+000,   0.00000000e+000,   0.00000000e+000,
          2.00000000e+000],
       [  1.99959046e+000,   4.09540058e-004,   4.09540058e-004,
          1.99959046e+000],
       [  1.99918061e+000,   8.19389420e-004,   8.19389420e-004,
          1.99918061e+000],
       ..., 
       [  0.00000000e+000,   0.00000000e+000,   0.00000000e+000,
          0.00000000e+000],
       [  0.00000000e+000,   1.63041663e-322,   0.00000000e+000,
          0.00000000e+000],
       [  0.00000000e+000,   0.00000000e+000,   0.00000000e+000,
          0.00000000e+000]]), z=array([[  0.00000000e+000,   0.00000000e+000,   0.00000000e+000,
          0.00000000e+000],
       [  0.00000000e+000,   0.00000000e+000,   0.00000000e+000,
          0.00000000e+000],
       [  0.00000000e+000,   0.00000000e+000,   0.00000000e+000,
          0.00000000e+000],
       ..., 
       [  2.52961611e-320,   3.03553933e-320,   2.52961611e-320,
          3.60739443e-313],
       [  2.75859610e-313,   6.36600002e-314,   1.39128886e-320,
          1.48539862e-313],
       [  2.52961611e-320,   1.27319904e-313,   6.36600002e-314,
          2.40313530e-320]]))], style={'axes': {'visible': True, 'label': {'color': 'black'}, 'ticklabel': {'color': 'black'}, 'color': 'black'}, 'box': {'visible': True}, 'background-color': 'white'}, tf=None, width=400, xlim=[-10.0, 10.0], ylim=[-10.0, 10.0], zlim=[0.0, 2.0]), HBox(children=(Play(value=0, interval=400, max=499), FloatSlider(value=0.0, max=499.0, step=1.0))))),

CPU times: user 313 ms, sys: 127 ms, total: 439 ms
Wall time: 299 ms
